# Исследование надёжности заёмщиков

**Заказчик** — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

**Входные данные от банка** — статистика о платёжеспособности клиентов.

## Обзор данных

In [1]:
import pandas as pd #импорт библиотеки pandas

In [2]:
df = pd.read_csv('/datasets/data.csv') #чтение файла с данными и сохранение в df
display(df.head(10)) #вывод на экран первых 10 строк таблицы df
df.info() #получение общей информации о таблице df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Итак, в таблице 12 столбцов. Тип данных в столбцах 'children', 'dob_years', 'education_id', 'family_status_id', 'debt' — `int64`. 
Тип данных в столбцах 'days_employed', 'total_income' — `float64`. Тип данных в столбцах 'education', 'family_status', 'gender', 'income_type', 'purpose' — `object`.

Согласно описанию данных:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

В названиях колонок нарушений стиля не выявлено.




Количество значений в столбцах `days_employed` и `total_income` отличается от остальных. Значит, в данных есть пропущенные значения. Также, в столбце `days_employed` представлены отрицательные и аномально большие значения, которые не могут соответствовать действительности, следовательно, в данных присутствуют артефакты (аномалии). Помимо этого, в столбце `total_income` данные представлены вещественными числами, что не совсем корректно отражает сами данные о ежемесячном доходе клиентов. И, наконец, в столбце `education` были замечены одни и те же значения, записанные по-разному, что свидетельствует о наличии дубликатов в данных.

В каждой строке таблицы - данные о клиентах банка. Часть колонок описывает самих клиентов: их пол, возраст, семейное положение, уровень образования, количество детей в семье. Остальные данные рассказывают об их платежеспособности: ежемесячный доход, тип занятости, трудовой стаж, наличие задолженностей по возврату кредитов. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез, однако встречаются пропуски. Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Обработка пропусков

In [3]:
print(df.isna().sum()) #подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**Вывод**

Были обнаружены пропущенные значения в двух столбцах: `days_employed` и `total_income`. Тип данных — `float64`. Пропуски в этих столбцах составляют 2174 / 21525 = 10,09% и могут напрямую повлиять на результаты исследования, поэтому их необходимо заполнить. 

Вероятная причина возникновения пропусков - клиенты сами не указали свои данные о трудовом стаже и ежемесячном доходе (возможно, из-за их отсутствия).  

Общий трудовой стаж в днях и ежемесячный доход являются количественными переменными. Пропуски в таких переменных следует заполнять характерными значениями, то есть значениями, характеризующими состояние выборки. Чтобы примерно оценить типичные значения выборки, годятся среднее арифметическое или медиана. В данном случае, лучшим решением будет заполнить пропуски медианным значением, поскольку среднее значение некорректно характеризует данные, когда есть значения, сильно выделяющиеся среди большинства.

Тем не менее, в столбце `days_employed` были также замечены отрицательные и аномально большие значения, которые не могут соответствовать действительности. Поэтому, прежде чем заполнять пропуски в этом столбце, необходимо привести данные в порядок, что будет сделано на следующем этапе.

Пропуски в столбце `total_income` следует заполнять медианными значениями в зависимости от типа занятости клиента, поскольку простое заполнение всех пропусков медианными значениями может существенно исказить реальные данные. 

In [4]:
df['total_income'] = df['total_income'].fillna(df.groupby(['income_type'])['total_income'].transform('median')) #замена пропущенных значений в столбце total_income на медианные в зависимости от типа занятости 
print(df.isna().sum()) #повторный подсчет пропусков для проверки

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64


Таким образом, пропущенные значения в столбце `total_income` были успешно заменены на медианные. Пропуски в столбце `days_employed` будут заполнены далее. 

### Аномалии

Аномальные значения были замечены в столбце `days_employed`. В нем присутствуют не только отрицательные, но и слишком большие значения, которые при пересчете в годы дают нереалистичный результат. К примеру, значение в строке 4 равно 340266.072047, что при пересчете в годы составит 340266 / 365 = 932 года. Также, следует проверить данные в столбцах `dob_years` и `children` 

В остальных столбцах значения категориальные, поэтому их проверка будет проведена позднее.

Начнем поиск аномалий со столбца `children`:

In [5]:
df['children'].value_counts() #просмотр уникальных значений в столбце children

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Среди уникальных значений есть два, значительно отличающихся от остальных: -1 и 20. Очевидно, что отрицательного количества детей в семье быть не может, и, вероятно, минус был поставлен по ошибке, а значит -1 следует преобразовать в 1. Похожая ситуация и с клиентами, у которых 20 детей. Безусловно, встречаются многодетные семьи с большим количеством детей, однако здесь это скорее всего ошибка в данных, поскольку наблюдается следующая закономерность: клиентов с 3 детьми - 330, с 4 детьми - 41, с 5 детьми - 9. Маловероятно, что клиентов с 20 детьми будет больше, чем клиентов с 4 и 5 детьми вместе взятых. К тому же, странно, что нет клиентов, например, с 6 или 10 детьми, а после 5 детей их число возрастает до 20. Исходя из вышесказанного, в этих 76 случаях, по всей видимости, по ошибке вместо 2 детей было указано 20.

In [6]:
df.loc[df['children'] == -1, 'children'] = 1 #замена значений, равных -1
df.loc[df['children'] == 20, 'children'] = 2 #замена значений, равных 20
            
df['children'].value_counts() #повторная проверка

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Все аномальные значения в столбце `children` были преобразованы. 

Далее, приступим к поиску и ликвидации аномалий в столбце `dob_years`. 

In [7]:
df['dob_years'].value_counts() #просмотр значений в столбце dob_years

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце `dob_years` также были обнаружены аномальные значения. В частности, у 101 клиента банка в графе "возраст" указано значение 0. Доля таких значений составляет 101 / 21525 = 0,46%. В отличие от столбца с количеством детей, здесь гораздо больше уникальных значений, включая те, которые заканчиваются на 0. К тому же, все остальные значения в столбце выглядят реалистично, в следствие чего "догадаться" о том, что подразумевалось под 0, не представляется возможным. Теоретически, эти данные можно просто отбросить, однако, более оптимальным вариантом будет замена значений, равных 0 на медианные.

Теперь, когда аномалии в столбце `dob_years` устранены, перейдем к изучению и ликвидации аномалий в столбце `days_employed`. Как было отмечено ранее, в данном столбце были замечены отрицательные и нереалистично большие значения, а также пропуски. Отрицательные значения преобразуем функцией abs(), нереалистично большие значения преобразуем в медианные. Критерий определения аномальных значений следующий: если стаж / 365 > 45 лет, то значение нужно заменить. Критерий был определен на том основании, что официальное трудоустройство возможно с 14-16 лет, а пенсионный возраст составляет 60-65 лет. Пропуски в столбце будут заполнены медианными значениями по столбцу. 

In [8]:
df['days_employed'] = df['days_employed'].abs() #преобразование отрицательных значений
df.loc[df['days_employed'] / 365 > 45, 'days_employed'] = df['days_employed'].median() #замена аномально больших значений медианными по столбцу
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median()) #заполнение пропусков медианными значениями по столбцу
print(df.isna().sum()) #повторная проверка на наличие пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Вывод**

Аномальные значения были замечены в столбцах `days_employed`, `dob_years` и `children`. Все они были заменены медианными значениями по соответствующим столбцам. Также, в столбце `days_employed` были заполнены пропуски, обнаруженные ранее. Вероятной причиной возникновения аномалий в столбцах `dob_years` и `children` могли стать ошибки со стороны людей, допущенные при заполнении данных, поскольку некоторые из них выглядят как опечатки (например, значение -1 в столбце `children` или значение 0 в столбце `dob_years`). А вот причина возникновения аномалий в столбце `days_employed` может иметь технический характер, поскольку в нем слишком много отрицательных значений, наряду с аномально большими. Вполне возможно, что ошибка возникла, например, в процессе считывания, копирования, смены формата или при записи данных.

### Замена типа данных

In [9]:
df.info() #проверка типов данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В столбцах `days_employed`, `dob_years`, `total_income` тип данных - вещественный. Это не слишком удобно для проведения дальнейших исследований, поэтому следует заменить тип данных в этих столбцах на целочисленный. Сделать это проще всего методом astype(), поскольку данные имеют тип float и уже очищены от аномальных значений, поэтому остается лишь указать необходимый тип данных. В данном случае - это 'int'.

In [10]:
for name in ['days_employed', 'dob_years', 'total_income']: #замена вещественного типа данных на целочисленный
    df[name] = df[name].astype('int')

df.info() #повторная проверка

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Было установлено, что в столбцах `days_employed`, `dob_years`, `total_income` тип данных - вещественный. Он был успешно заменен на целочисленный методом astype().

### Обработка дубликатов

In [11]:
print(df.duplicated().sum()) #подсчет явных дубликатов

54


Были обнаружены 55 явных дубликатов. Возможные причины возникновения дубликатов — неправильное соединение данных из разных источников, ошибки со стороны человека при занесении информации. Дубликаты следует удалить. Сделать это легче всего методом drop_duplicates().

In [12]:
df = df.drop_duplicates().reset_index(drop=True) #удаление явных дубликатов (с удалением старых индексов и формированием новых)
print(df.duplicated().sum()) #повторный подсчет явных дубликатов для проверки

0


Все явные дубликаты были удалены. Далее, следует проверить данные на наличие неявных дубликатов. 

In [13]:
df['education'].value_counts() #просмотр уникальных значений в столбце education

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

В столбце `education` были обнаружены одинаковые строки, отличающиеся регистром букв. Их следует привести к одному регистру.

In [14]:
df['education'] = df['education'].str.lower() #приведение всех символов в строках к нижнему регистру
df['education'].value_counts() #повторная проверка

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим и другие столбцы.

In [15]:
df['purpose'].value_counts() #просмотр уникальных значений в столбце purpose

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

В столбце `purpose` присутствует большое количество повторяющихся по смыслу строк, записанных по-разному. Все они могут быть отнесены к одной из четырех категорий: 'операции с автомобилем', 'операции с недвижимостью', 'проведение свадьбы', 'получение образования'.  Оригинальные значения можно сохранить, а разделить по категориям и вынести их в отдельный столбец позднее, на этапе категоризации данных.

На всякий случай, следует проверить остальные столбцы, ведь в них также могут оказаться неявные дубликаты или иные проблемы.

In [16]:
print(df['family_status'].value_counts()) #просмотр уникальных значений в столбце family_status
print(df['gender'].value_counts()) #просмотр уникальных значений в столбце gender
print(df['income_type'].value_counts()) #просмотр уникальных значений в столбце income_type

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64
F      14189
M       7281
XNA        1
Name: gender, dtype: int64
сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


В столбце `family_status` в значении 'Не женат / не замужем' есть заглавная буква 'Н'. Она никак не влияет на сами данные, однако ее все равно лучше перевести в нижний регистр. В столбце `gender` было обнаружено значение 'XNA'. Судя по всему, это текст-заполнитель, который также не влияет на ход исследования, но для единообразия данных вместо него можно указать, например, 'М'. В столбце `income_type` никаких проблем не обнаружено.

In [17]:
df['family_status'] = df['family_status'].str.lower() #перевод буквы Н в нижний регистр
#df.loc[df['gender'] == 'XNA', 'gender'] = 'M' #замена текста-заполнителя

**Вывод**

Были обнаружены 55 явных дубликатов. Они были удалены методом drop_duplicates(). Также, были обнаружены неявные дубликаты, различающиеся регистром букв в  столбце `education`. Все они были приведены к нижнему регистру. В столбце `purpose` оказалось большое количество повторяющихся по смыслу строк, записанных по-разному. Было принято решение оставить оригинальные формулировки целей кредитов, но при этом разделить их по категориям и вынести эти категории в отдельный столбец на этапе категоризации данных. Помимо этого, были обнаружены и устранены несколько мелких недочетов в столбцах `family_status` и `gender`.

### Категоризация данных

Для того, чтобы результаты исследования были более наглядными, следует провести категоризацию данных - объединение избранных данных в произвольные группы по заданному критерию. В ходе предыдущего этапа был обнаружен столбец `purpose`, значения в котором можно разбить на 4 категории: 'операции с автомобилем', 'операции с недвижимостью', 'проведение свадьбы', 'получение образования'. Сделано это будет с помощью созданной функции purpose_categories. Критерий следующий: если в ячейке есть значение 'авто', то значение в той же строке в столбце 'purpose_category' - 'операции с автомобилем'. Если в ячейке есть значение 'жил' или 'недв' - то 'операции с недвижимостью'. Если есть значение 'свадьб' - то 'проведение свадьбы'. Если есть значение 'образован' - то 'получение образования'. Такой критерий обоснован тем, что во всех ячейках данного столбца встречаются производные одних и тех же слов, на основании лемм которых и проводится разделение по категориям.

In [18]:
def purpose_categories(purpose): #функция для категоризации данных
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жил' in purpose or 'недв' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования' 

    
df['purpose_category'] = df['purpose'].apply(purpose_categories) #применение функции к столбцу purpose и сохранение значений в столбец purpose_category
print(df['purpose_category'].value_counts()) #проверка успешности выполнения функции

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64


Еще один столбец, в котором следует категоризовать данные для наглядности - `total_income`. Здесь также будет применена собственная функция. Принцип ее работы аналогичен предыдущей, различаются только критерии. В данном случае, клиенты будут разбиты по категориям в зависимости от их ежемесячного дохода: 0–30000 — E; 30001–50000 — D; 50001–200000 — C; 200001–1000000 — B; 1000001 и выше — A.

In [19]:
def income_categories(total_income): #функция для категоризации данных
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000 and total_income >= 30001:
        return 'D'
    if total_income <= 200000 and total_income >= 50001:
        return 'C'
    if total_income <= 1000000 and total_income >= 200001:
        return 'B'
    if total_income >= 1000001:
        return 'A'
 
 
df['total_income_category'] = df['total_income'].apply(income_categories) #применение функции к столбцу total_income и сохранение значений в столбец total_income_category
df['total_income_category'].value_counts() #проверка успешности выполнения функции

C    16032
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

**Вывод**

Проведена категоризация данных, находящихся в столбцах `purpose` и `total_income`. Для этого были написаны две соответствующие функции и созданы два столбца, в которые были записаны результаты категоризации. На данном этапе, данные приведены в приемлемое состояния для проведения анализа и проверки гипотез.

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [20]:
pd.pivot_table(df, index='children', columns='debt', values='total_income', aggfunc='mean') #создание сводной таблицы, отражающей зависимость между наличием детей и возвратом кредита в срок
df.groupby('children').agg({'debt' : ['mean', 'sum', 'count']})

debt             
              mean   sum  count
children                       
0         0.075353  1063  14107
1         0.091639   445   4856
2         0.094925   202   2128
3         0.081818    27    330
4         0.097561     4     41
5         0.000000     0      9

**Вывод**

Общее число клиентов — 21470 чел., из них 14106 чел. (65,7%) — `без детей`. Общее число должников — 1741 чел., среди них должников `без детей` — 1063 чел. (61,05%).

Наибольшая доля должников наблюдается среди клиентов `с 4 детьми` (9,75%). Затем идут клиенты `с 2 детьми` (9,49%) и клиенты `с 1 ребенком` (9,16%). Наименьшая доля среди тех клиентов, у которых вообще `нет детей` (7,53%), а также среди клиентов `с 3 детьми` (8,18%). В выборке не оказалось ни одного клиента-должника `с 5 детьми` (0,00%), поэтому делать какие-либо выводы о данной категории некорректно. Если же взглянуть на абсолютные показатели, то в выборке в целом очень мало клиентов `с 4 и 5 детьми` (41 и 9 чел. соответственно), что не позволяет провести достаточно точный анализ данных по ним. При этом, данных об остальных категориях клиентов более чем достаточно для анализа. В целом, наиболее платежеспособными представляются клиенты `без детей`. Их численность наибольшая (14106 чел.), в то время как процент должников — наименьший (7,53%). Среди клиентов `с детьми` доля должников больше (в среднем, 8-9%), однако самих клиентов существенно меньше (7364 чел. или 34,3%, если объединить всех клиентов с детьми).

Согласно представленным данным, можно сделать вывод о том, что вероятность возврата кредита в срок среди клиентов `без детей` примерно на 1,5% выше, чем среди клиентов `с детьми`.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [21]:
pd.pivot_table(df, index='family_status', columns='debt', values='total_income', aggfunc='mean') #создание сводной таблицы, отражающей зависимость между семейным положением и возвратом кредита в срок
df.groupby('family_status').agg({'debt' : ['mean', 'sum', 'count']})

debt            
                           mean  sum  count
family_status                              
в разводе              0.071130   85   1195
вдовец / вдова         0.065693   63    959
гражданский брак       0.093202  388   4163
женат / замужем        0.075421  931  12344
не женат / не замужем  0.097509  274   2810

**Вывод**

Общее число клиентов — 21470 чел., из них 12344 чел. (57,49%) — `женаты / замужем`. Общее число должников — 1741 чел., среди них должников, имеющих семейный статус `женат / замужем` — 931 чел. (53,47%).

Наибольшая доля должников наблюдается среди клиентов со статусом `не женат / не замужем` (9,75%). Затем идут клиенты, состоящие `в гражданском браке` (9,32%). Наименьшая доля клиентов, имевших задолженность по кредитам — среди имеющих семейный статус `вдовец / вдова` (6,56%). Среди клиентов, находящихся `в разводе` должников около 7,11%, среди тех, кто `женат / замужем` — 7,54%.    

Исходя из представленных данных, затруднительно сделать однозначные выводы о зависимости между семейным положением и возвратом кредита в срок. С одной стороны, можно отметить, что 3 категории клиентов, состоящих в официальном браке в данный момент или ранее (`женат / замужем`, `в разводе`, `вдовец / вдова`) составляют большую часть клиентов банка (67,52%), при этом имея более низкий процент должников, чем оставшиеся две категории (`не женат / не замужем`, `гражданский брак`). С другой стороны, примечателен тот факт, что наличие / отсутствие супруга / супруги, судя по всему, не оказывает прямого влияния на факт возврата кредита в срок. Так, например, возьмем для сравнения две категории: `не женат / не замужем` и `вдовец / вдова`. Для более справедливого сравнения, к последней категории можно также добавить категорию `в разводе`, тогда численность сравниваемых категорий будет примерно сопоставима (2810 чел. и 2154 чел.). В обоих случаях, наблюдается отсутствие супруга / супруги, однако среди клиентов со статусом `не женат / не замужем` доля должников существенно выше, чем среди клиентов со статусами `в разводе` и `вдовец / вдова` (примерно на 2,5%). Таким образом, можно предположить, что в первую очередь на возврат кредита в срок влияет не столько наличие супруга / супруги, сколько сам факт состояния в официальном браке. Возможно, причиной тому служат какие-либо юридические особенности и ответственность, связанные с браком.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [22]:
pd.pivot_table(df, index='total_income_category', columns='debt', values='total_income', aggfunc='mean') #создание сводной таблицы, отражающей зависимость между уровнем дохода и возвратом кредита в срок
df.groupby('total_income_category').agg({'debt' : ['mean', 'sum', 'count']})

debt             
                           mean   sum  count
total_income_category                       
A                      0.080000     2     25
B                      0.070607   356   5042
C                      0.084830  1360  16032
D                      0.060000    21    350
E                      0.090909     2     22

**Вывод**

Клиенты банка были распределены по категориям согласно их уровню дохода следующим образом: 0–30000 — `E`; 30001–50000 — `D`; 50001–200000 — `C`; 200001–1000000 — `B`; 1000001 и выше — `A`.

Общее число клиентов — 21470 чел., из них 16031 чел. (74,66%) относятся к категории `C`. Общее число должников — 1741 чел., среди них должников, имеющих категорию `C` — 1360 чел. (84,83%).

Наибольшая доля должников наблюдается среди клиентов из категории `E` (9,09%). Затем идут клиенты из категории `C` (8,48%). Наименьшая доля клиентов, имевших задолженность по кредитам — в категории `D` (6,00%). Среди клиентов категории `B`, должников около 7,06%, среди тех, кто относится к категории `A` — 8,00%. Однако, следует отметить, что в выборке представлено всего 25 клиентов категории `A` и 22 клиента категории `E`, поэтому общие выводы по данным категориям могут быть недостаточно точными.      

Исходя из представленных данных, затруднительно сделать вывод о существовании какой-либо очевидной зависимости между ежемесячным доходом и вероятностью возврата кредита в срок. Так, например, должников в категории `A` примерно на 1% больше, чем в категории `B` и на 2% больше, чем в категории `D`. Это может быть, например, связано с тем, что клиенты категории `A` берут в кредит существенно большие суммы, которые сложнее вернуть. Однако, это лишь косвенное следствие увеличения дохода, зависящее скорее от прочих факторов (размеров сумм, целей кредитов и др.), но здесь нельзя выделить прямую зависимость от ежемесячного дохода самого клиента. В качестве наиболее платежеспособных можно отметить категории `D` и `B`, а наименее платежеспособной является категория `E`.

- Как разные цели кредита влияют на его возврат в срок?

In [23]:
pd.pivot_table(df, index='purpose', columns='debt', values='total_income', aggfunc='mean') #создание сводной таблицы, отражающей зависимость между целями кредита и его возвратом в срок
df.groupby('purpose_category').agg({'debt' : ['mean', 'sum', 'count']})

debt            
                              mean  sum  count
purpose_category                              
операции с автомобилем    0.093547  403   4308
операции с недвижимостью  0.072314  782  10814
получение образования     0.092177  370   4014
проведение свадьбы        0.079657  186   2335

**Вывод**

Общее число клиентов — 21470 чел., из них 10814 чел. (50,36%) взяли кредиты для проведения `операций с недвижимостью`. Общее число должников — 1741 чел., среди них 782 чел. взяли кредиты для проведения `операций с недвижимостью` (44,91%).

Наибольшая доля должников наблюдается среди клиентов, взявших кредиты для осуществления `операций с автомобилем` (9,35%) и с целью `получения образования` (9,21%). Затем идут клиенты, взявшие кредиты для `проведения свадьбы` (7,96%). Наименьшая доля должников — среди клиентов, взявших кредиты на `операции с недвижимостью`. 

В целом, можно сделать вывод о том, что клиенты, берущие кредиты на `операции с недвижимостью` являются самой платежеспособной и многочисленной группой. А вот среди клиентов, чьей целью является `получение образования` и осуществление `операций с автомобилем` отмечается примерно равное количество должников (9,21-9,35%), что примерно на 2% больше, чем среди клиентов с целью `операции с недвижимостью`.   

## Общий вывод

В ходе проведенного исследования были проанализированы полученные от кредитного отдела банка статистические данные о платёжеспособности клиентов. Цель исследования — анализ влияния семейного положения и количества детей клиента на факт погашения кредита в срок.

В полученных входных данных наблюдался ряд проблем, таких как: пропуски, дубликаты и аномальные значения. Все проблемы были устранены в процессе исследования, а данные — приведены в максимально пригодное для проведения анализа состояние.

Были проверены несколько гипотез и даны ответы на следующие вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
    
    Первая гипотеза подтвердилась. Согласно представленным данным, был сделан вывод о том, что вероятность возврата кредита в срок среди клиентов без детей примерно на 1,5% выше, чем среди клиентов с детьми.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?
    
    Вторая гипотеза также подтвердилась. Было установлено, что в первую очередь на возврат кредита в срок влияет не столько наличие супруга / супруги, сколько сам факт состояния в официальном браке.


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

    Третья гипотеза не подтвердилась. Исходя из представленных данных, оказалось затруднительно сделать вывод о существовании какой-либо очевидной зависимости между ежемесячным доходом и вероятностью возврата кредита в срок. В качестве наиболее платежеспособных были выделены категории клиентов D и B, а наименее платежеспособной — категория E.


- Как разные цели кредита влияют на его возврат в срок?

    Было установлено, что клиенты, берущие кредиты для операций с недвижимостью являются самой платежеспособной и многочисленной группой. Категории клиентов, чьими целями является получение образования и операции с автомобилем оказались примерно равны и сопоставимы по всем показателям, при этом являясь наименее платежеспособными из всех представленных групп.
    
В ходе исследования было установлено, что некоторые категории клиентов были недостаточно широко представлены в выборке, поэтому некоторые выводы могут оказаться не совсем корректными и должны быть пересмотрены при условии изменения входных данных.

Таким образом, в ходе проведенного исследования все поставленные задачи были выполнены.